In [1]:
import sys, os, time
# %load_ext autoreload
# %autoreload 2
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
os.environ['FOUNDATION_SAVE_DIR'] = '/is/ei/fleeb/workspace/chome/trained_nets'
os.environ['FOUNDATION_DATA_DIR'] = '/is/ei/fleeb/workspace/local_data'
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
%matplotlib notebook
# %matplotlib tk
import matplotlib.pyplot as plt
import imageio
import seaborn as sns
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data

np.set_printoptions(linewidth=120, suppress=True)

In [2]:
M = train.Run_Manager(tbout='/is/ei/fleeb/workspace/trained_nets/managed')
M.clear_links()
M.start_tb()

/is/ei/fleeb/workspace/trained_nets/managed is available to view runs on tensorboard


/is/ei/fleeb/workspace/foundation/foundation/train/config.py:58: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)


Found 130 runs
Tensorboard started: http://localhost:6006/


In [3]:
#M.show()
M.clear_filters().select(model=None, dataset='3dscr')
M.show()

  0 - 3dscr-dvae_0078-5920873-7_191213-202209
  1 - 3dscr-dvae_0082-5923472-07_191218-100110
  2 - 3dscr-dwae_0078-5920873-6_191213-202209
  3 - 3dscr-dwae_0082-5923472-06_191218-100109
  4 - 3dscr-fdvae_0078-5920873-9_191213-202210
  5 - 3dscr-fdvae_0082-5923472-09_191218-100109
  6 - 3dscr-fdvae_0082-5923472-11_191218-100110
  7 - 3dscr-fdwae_0078-5920873-8_191213-202213
  8 - 3dscr-fdwae_0082-5923472-08_191218-100110
  9 - 3dscr-fdwae_0082-5923472-10_191218-100109
 10 - 3dscr-fvae_0078-5920873-3_191213-202212
 11 - 3dscr-fvae_0082-5923472-03_191218-100108
 12 - 3dscr-fwae_0078-5920873-2_191213-202209
 13 - 3dscr-fwae_0082-5923472-02_191218-100114
 14 - 3dscr-vae_0078-5920873-1_191213-202211
 15 - 3dscr-vae_0082-5923472-01_191218-100110
 16 - 3dscr-wae_0078-5920873-0_191213-202222
 17 - 3dscr-wae_0082-5923472-00_191218-100123
 18 - 3dscr-wgan_0078-5920873-4_191213-202211
 19 - 3dscr-wgan_0080-5920891-0_191214-090741
 20 - 3dscr-wgan_0080-5920891-1_191214-090742
 21 - 3dscr-wgan_0082-

In [4]:
ignore = None
# ignore = ['disc_steps']
M.show_unique(ignore)

0) 3dscr-dvae_0078-5920873-7_191213-202209
	model.disc_steps - 1 (5)

1) 3dscr-dvae_0082-5923472-07_191218-100110
	model.disc_steps - 1 (5)

2) 3dscr-dwae_0078-5920873-6_191213-202209
	model.disc_steps - 1 (5)

3) 3dscr-dwae_0082-5923472-06_191218-100109
	model.disc_steps - 1 (5)

4) 3dscr-fdvae_0078-5920873-9_191213-202210
	model.disc_steps - 1 (5)

5) 3dscr-fdvae_0082-5923472-09_191218-100109
	model.disc_steps - 1 (5)

6) 3dscr-fdvae_0082-5923472-11_191218-100110
	model.disc_steps - 1 (5)
	model.prob - 0.5 (1)

7) 3dscr-fdwae_0078-5920873-8_191213-202213
	model.disc_steps - 1 (5)

8) 3dscr-fdwae_0082-5923472-08_191218-100110
	model.disc_steps - 1 (5)

9) 3dscr-fdwae_0082-5923472-10_191218-100109
	model.disc_steps - 1 (5)
	model.prob - 0.5 (1)

10) 3dscr-fvae_0078-5920873-3_191213-202212
	model.disc_steps - 1 (5)

11) 3dscr-fvae_0082-5923472-03_191218-100108
	model.disc_steps - 1 (5)

12) 3dscr-fwae_0078-5920873-2_191213-202209
	model.disc_steps - 1 (5)

13) 3dscr-fwae_0082-5923472-02

In [5]:
M.link('{idx}__{model}__{unique}')

In [6]:
# M.link('{name}__{unique}')

In [6]:
# os.listdir(M.tbout)

In [9]:
M.clear_links()

In [8]:
M.tbout

'/is/ei/fleeb/workspace/trained_nets/managed'

In [18]:
import json

In [19]:
json.loads('[10,2,3]')

[10, 2, 3]

In [ ]:
saveroot = os.environ['FOUNDATION_SAVE_DIR']
saveroot

In [ ]:
for name in os.listdir(saveroot):
    path = os.path.join(saveroot, name)
    if 'config.yml' in os.listdir(path):
        print(name,end=' ')
        with open(os.path.join(path, 'config.yml'), 'r') as f:
            data = yaml.load(f)
        break
#         if 'parents' in data:
#             del data['parents']
#             with open(os.path.join(path, 'config.yml'), 'w') as f:
#                 yaml.dump(data, f)
#             print('fixed')
#         else:
#             print()

In [ ]:
data['parents']